# Tutorial 1. The Measurement Control 

This tutorial covers basic usage of PycQED focusing on running basic experiments using `MeasurementControl`. 
The `MeasurementControl` is the main `Instrument` in charge of running any experiment. It takes care of saving the data in a standardized format as well as live plotting of the data during the experiment. 
`PycQED` makes a distinction between `soft`(ware) controlled measurements and `hard`(ware) controlled measurements. 

In a `soft` measurement `MeasurementControl` is in charge of the measurement loop and consecutively sets and gets datapoints. A `soft` measurement can be 1D, 2D or higher dimensional and also supports adaptive measurements in which the datapoints are determined during the measurement loop. 

In a `hard` measurement the hardware (such as an AWG or a central controller) is in charge of the measurement loop. In this case, the datapoints to be acquired are determined before the experiment starts and are precompiled into the hardware which is then armed and starts acquisition. In a `hard` measurement `MeasurementControl` does not take care of the measurement loop but still takes care of the data storage and live plotting of the experiment. 

## import required modules

In [1]:
import pycqed as pq
import numpy as np
from pycqed.measurement import measurement_control
from pycqed.measurement.sweep_functions import None_Sweep
import pycqed.measurement.detector_functions as det
from qcodes import station
station = station.Station()


/Users/adriaanrol/GitHubRepos/DiCarloLab_Repositories/PycQED_py3/data
Data directory set to: /Users/adriaanrol/GitHubRepos/DiCarloLab_Repositories/PycQED_py3/data
Could not import msvcrt (used for detecting keystrokes)
/Users/adriaanrol/GitHubRepos/DiCarloLab_Repositories/PycQED_py3/data


/usr/local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Creating an instance of `MeasurementControl`

Measurements are controlled through the `MeasurementControl` usually instantiated as `MC`

In [2]:
MC = measurement_control.MeasurementControl('MC',live_plot_enabled=True, verbose=True)
MC.station = station
station.add_component(MC)

'MC'

The `InstrumentMonitor` can be used to see the parameters of any instrument connected to the station and updates during the loop initiated by `MeasurementControl`. 

In [3]:
from pycqed.instrument_drivers.virtual_instruments import instrument_monitor as im 
IM = im.InstrumentMonitor('IM', station)
station.add_component(IM)
# Link the instrument monitor to the MC so that it gets updated in the loop
MC.instrument_monitor('IM')
IM.update()

In [7]:
IM.update_interval(.1)
IM.update()

## Create instruments used in the experiment 
Let's start by creating a dummy instrument called MockParabola. 


In [8]:
from pycqed.instrument_drivers.physical_instruments.dummy_instruments import DummyParHolder
dummy_instrument = DummyParHolder('dummy_instrument')
station.add_component(dummy_instrument)

'dummy_instrument'

## A 1D hard measurement 
A hard measurement is a measurement where the data acquisition loop happens in the **hard**ware.

In [9]:
MC.soft_avg(15)
MC.persist_mode(True)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_points(np.linspace(0, 10, 30))
MC.set_detector_function(det.Dummy_Detector_Hard(noise=0.5, delay=.02))
dat = MC.run('dummy_hard')
data_set = dat['dset']

Starting measurement: dummy_hard
Sweep function: None_Sweep
Detector function: Dummy_Detector_Hard
 100% completed 	elapsed time: 12.7s 	time left: 0.0s


By setting persist_mode = True we can see a copy of the last measurements

In [10]:

MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_points(np.linspace(0, 10, 30))
MC.set_detector_function(det.Dummy_Detector_Hard(noise=0.5, delay=.02))
dat2 = MC.run('dummy_hard persistent')
data_set2 = dat2['dset']

Starting measurement: dummy_hard persistent
Sweep function: None_Sweep
Detector function: Dummy_Detector_Hard
 100% completed 	elapsed time: 2.4s 	time left: 0.0s


# A simple 1D soft measurement 
A soft measurement is a a measurement where the data acquisition loop occurs in the **soft**ware

In [11]:
dummy_instrument.x(145/134545)
IM.update()

In [12]:
dummy_instrument.delay(.01)
MC.soft_avg(15)
MC.set_sweep_function(dummy_instrument.x)
MC.set_sweep_points(np.linspace(-1,1,30))
dummy_instrument.noise(1)

MC.set_detector_function(dummy_instrument.parabola)
dat = MC.run('1D test')
data_set = dat['dset']

# the second plot will also show the first line
MC.set_sweep_function(dummy_instrument.x)
MC.set_sweep_points(np.linspace(-1,1,30))

dat2= MC.run('1D test-persist')
data_set2 = dat2['dset']

Starting measurement: 1D test
Sweep function: x
Detector function: parabola
 97% completed 	elapsed time: 7.4s 	time left: 0.2ss

 100% completed 	elapsed time: 7.6s 	time left: 0.0s
Starting measurement: 1D test-persist
Sweep function: x
Detector function: parabola
 100% completed 	elapsed time: 7.9s 	time left: 0.0s


In [11]:
dummy_instrument.delay(.01)
MC.soft_avg(15)
MC.set_sweep_function(dummy_instrument.x)
MC.set_sweep_points(np.linspace(-1,1,30))

MC.set_detector_function(det.Dummy_Detector_Soft())
dat = MC.run('1D test')
data_set = dat['dset']


Starting measurement: 1D test
Sweep function: x
Detector function: Dummy_Detector_Soft
 100% completed 	elapsed time: 1.3s 	time left: 0.0s


You can play around a bit with the options in the MC: 

In [13]:
MC.persist_mode(True) # Turns on and off persistent plotting
MC.verbose(True)
MC.plotting_interval(.2)
MC.live_plot_enabled(True)

# A simple 2D measurement

In [13]:
dummy_instrument.delay(.0001)
MC.soft_avg(4)

sweep_pts = np.linspace(-2, 2, 30)
sweep_pts_2D = np.linspace(-2, 2, 5)

MC.set_sweep_function(dummy_instrument.x)
MC.set_sweep_function_2D(dummy_instrument.y)
MC.set_sweep_points(sweep_pts)
MC.set_sweep_points_2D(sweep_pts_2D)
MC.set_detector_function(dummy_instrument.parabola)
dat=MC.run('test', mode='2D')
data_set = dat['dset']


Starting measurement: test
Sweep function 0: x
Sweep function 1: Sweep_function
Detector function: parabola
 100% completed 	elapsed time: 8.1s 	time left: 0.0ss


# 2D combinatioin of a hard inner and soft outer loop
The hard inner loop returns 30 values 

In [14]:
MC.soft_avg(1)
sweep_pts = np.linspace(0, 10, 30)
sweep_pts_2D = np.linspace(0, 10, 30)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_function_2D(None_Sweep(sweep_control='soft'))
MC.set_sweep_points(sweep_pts)
MC.set_sweep_points_2D(sweep_pts_2D)
MC.set_detector_function(det.Dummy_Detector_Hard(delay=.05, noise=.1))
dat = MC.run('2D_hard', mode='2D')
data_set = dat['dset']

Starting measurement: 2D_hard
Sweep function 0: None_Sweep
Sweep function 1: None_Sweep
Detector function: Dummy_Detector_Hard
 100% completed 	elapsed time: 7.0s 	time left: 0.0s


## A Hard measurement that uses soft averaging

The number of soft_averages determines how many times the experiment will be performed. 
Only the averaged data is plotted and saved. 
The number of soft-averages can be set as a parameter of the Measurement Control. 

Will first implement it for 1D hard sweeps (easier) and then follow for combinations of hard and soft sweeps. 

In [21]:
MC.soft_avg(4)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_points(np.linspace(0, 10, 30))
MC.set_detector_function(det.Dummy_Detector_Hard(noise=1.5, delay=.02))

dat = MC.run('dummy_hard')
data_set = dat['dset']

Starting measurement: dummy_hard
Sweep function: None_Sweep
Detector function: Dummy_Detector_Hard
 100% completed 	elapsed time: 1.2s 	time left: 0.0s


# 2D soft averaging

In [22]:
MC.soft_avg(10)
sweep_pts = np.linspace(0, 10, 30)
sweep_pts_2D = np.linspace(0, 10, 5)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_function_2D(None_Sweep(sweep_control='soft'))
MC.set_sweep_points(sweep_pts)
MC.set_sweep_points_2D(sweep_pts_2D)
MC.set_detector_function(det.Dummy_Detector_Hard(noise=1.5, delay=.001))

dat = MC.run('dummy_hard_2D', mode='2D')
data_set = dat['dset']

Starting measurement: dummy_hard_2D
Sweep function 0: None_Sweep
Sweep function 1: None_Sweep
Detector function: Dummy_Detector_Hard
 100% completed 	elapsed time: 15.0s 	time left: 0.0s



## Starting an adaptive measurement 
This example does a 2D optimization over the mock parabola

In [17]:
dummy_instrument.delay(.05)

In [22]:
dummy_instrument.noise(2)

In [24]:
from pycqed.measurement.optimization import nelder_mead
MC.soft_avg(1)
dummy_instrument
MC.set_sweep_functions([dummy_instrument.x, dummy_instrument.y])
MC.set_adaptive_function_parameters({'adaptive_function':nelder_mead, 
                                    'x0':[-5,-5], 'initial_step': [2.5, 2.5]})
dummy_instrument.noise(2)
MC.set_detector_function(dummy_instrument.parabola)
dat = MC.run('1D test', mode='adaptive')
data_set = dat['dset']

Starting measurement: 1D test
Sweep function 0: x
Sweep function 1: y
Detector function: parabola
Acquired 58 points, 	elapsed time: 14.6s

For a more advanced example of adaptive measurements see "Tutorial: Measurement Control - adaptive sampling". 